In [10]:
!pip install yfinance

https://pypi.org/project/yfinance/ 

In [11]:
import numpy as np
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt

In [ ]:
# def upload_prices_COTdates()


# def make_futureprice_matrix()

In [12]:
# Файл с датировкой отчетов СОТ
dates_url = 'https://raw.githubusercontent.com/ForestTiger-GH/COTRA/main/Даты%20отчетов%20СОТ.txt'
dates = pd.read_csv('https://raw.githubusercontent.com/ForestTiger-GH/COTRA/main/COTR_Dates.txt', parse_dates=True)

In [13]:
dates['Date'] = pd.to_datetime(dates['Date'])
# dates['fDate'] = dates['Date']

In [14]:
# Commodities
Wheat_SRW = yf.Ticker('ZW=F').history(period="max")
Wheat_SRW.index.name = 'Index'
Wheat_SRW['Date'] = Wheat_SRW.index
XXX = Wheat_SRW.append(dates)
Wheat_SRW = XXX.sort_values(by=['Date', 'Close'], ascending=True).ffill().drop_duplicates(subset=['Date']).merge(dates['Date'], how='right')

Soybeans = yf.Ticker('ZS=F').history(period="max")
Soybeans.index.name = 'Index'

In [15]:
# Financial
Canadian_Dollar = yf.Ticker('6C=F').history(period="max")
Canadian_Dollar.index.name = 'Index'
Canadian_Dollar['Date'] = Canadian_Dollar.index
XXX = Canadian_Dollar.append(dates)
Canadian_Dollar = XXX.sort_values(by=['Date', 'Close'], ascending=True).ffill().drop_duplicates(subset=['Date']).merge(dates['Date'], how='right')

British_Pound = yf.Ticker('6B=F').history(period="max")
British_Pound.index.name = 'Index'
British_Pound['Date'] = British_Pound.index

In [18]:
Wheat_SRW['Week_01'] = Wheat_SRW['Close'].shift(-1)

In [19]:
Wheat_SRW

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,Week_01
0,485.25,489.00,475.75,476.75,31447.0,0.0,0.0,2010-03-23,472.00
1,464.75,473.75,462.25,472.00,34897.0,0.0,0.0,2010-03-30,463.50
2,454.00,470.00,448.75,463.50,46216.0,0.0,0.0,2010-04-06,476.00
3,467.00,476.50,463.50,476.00,64576.0,0.0,0.0,2010-04-13,486.00
4,469.50,487.25,468.50,486.00,36954.0,0.0,0.0,2010-04-20,478.25
...,...,...,...,...,...,...,...,...,...
531,517.00,517.25,506.75,508.00,76980.0,0.0,0.0,2020-06-02,504.50
532,512.00,514.00,500.75,504.50,105093.0,0.0,0.0,2020-06-09,496.00
533,506.50,507.25,491.00,496.00,67274.0,0.0,0.0,2020-06-16,486.00
534,485.25,488.75,479.25,486.00,38396.0,0.0,0.0,2020-06-23,490.00
